# Projet 5: Catégorisez automatiquement des questions

## Partie 2: Tests des modèles

In [1]:
#importation des librairies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
import ast
#import re

#binarisation des tags
from sklearn.preprocessing import MultiLabelBinarizer

#librairies pour le tfidf - extraction d'information du texte
from sklearn.feature_extraction.text import TfidfVectorizer

#Pour LDA
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn

#pour NMF
from sklearn.decomposition import NMF

#pour les modèles supervisés
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

#Traitement des données
#from bs4 import BeautifulSoup #extraction du texte
import string
import nltk #traitement du texte
nltk.download('punkt')
#nltk.tokenize.word_tokenize #tokenization
#w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
#nltk.download('stopwords')
#from nltk.corpus import stopwords #stop words
#stop_words = stopwords.words('english') 
#nltk.download('wordnet')
#from nltk.stem import WordNetLemmatizer #lemmatisation
#lemmatizer = WordNetLemmatizer()
#from nltk.stem import PorterStemmer #stemming porter
#porter = PorterStemmer()
#from nltk.stem import LancasterStemmer #stemming lancaster
#lancaster=LancasterStemmer()

#importation des librairies - visualisation
#from sklearn.feature_extraction.text import CountVectorizer
#import collections


C:\Users\alice\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\alice\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
C:\Users\alice\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\Users\alice\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequ

True

In [2]:
#importation du dataset
df = pd.read_csv('clean_dataframe_final.csv')

In [3]:
df.shape

(38393, 5)

In [4]:
df.head()

,Text,TagsTokens,text_lemmatized,text_stemm_porter,text_stemm_lancaster
0,c# 50 await keyword listed operator rather mod...,['c#'],"['c#', '50', 'await', 'keyword', 'listed', 'op...","['c#', '50', 'await', 'keyword', 'list', 'oper...","['c#', '50', 'await', 'keyword', 'list', 'op',..."
1,configure mercurial use winmerge merges cygwin...,['mercurial'],"['configure', 'mercurial', 'use', 'winmerge', ...","['configur', 'mercuri', 'use', 'winmerg', 'mer...","['config', 'merc', 'us', 'winmerg', 'merg', 'c..."
2,whats best way go photoshop mockup semantic ht...,"['html', 'css']","['whats', 'best', 'way', 'go', 'photoshop', 'm...","['what', 'best', 'way', 'go', 'photoshop', 'mo...","['what', 'best', 'way', 'go', 'photoshop', 'mo..."
3,lookup data book barcode number building world...,['language-agnostic'],"['lookup', 'data', 'book', 'barcode', 'number'...","['lookup', 'data', 'book', 'barcod', 'number',...","['lookup', 'dat', 'book', 'barcod', 'numb', 'b..."
4,embeddable webkit component windows c# develop...,"['.net', 'com']","['embeddable', 'webkit', 'component', 'window'...","['embedd', 'webkit', 'compon', 'window', 'c#',...","['embed', 'webkit', 'compon', 'window', 'c#', ..."


In [5]:
df['TagsTokens']

0                            ['c#']
1                     ['mercurial']
2                   ['html', 'css']
3             ['language-agnostic']
4                   ['.net', 'com']
                    ...            
38388              ['php', 'mysql']
38389               ['html', 'css']
38390    ['c#', '.net', 'generics']
38391               ['html', 'xml']
38392                    ['winapi']
Name: TagsTokens, Length: 38393, dtype: object

## 1. Tests non supervisés sur données avec lemmatisation

### 1.1 Préparation des données

#### 1.1.1 Construction du dataset

In [6]:
#construction du dataset
cols = ['TagsTokens','text_lemmatized']
df_lemm = df[cols]

In [7]:
df_lemm.shape

(38393, 2)

In [8]:
df_lemm

,TagsTokens,text_lemmatized
0,['c#'],"['c#', '50', 'await', 'keyword', 'listed', 'op..."
1,['mercurial'],"['configure', 'mercurial', 'use', 'winmerge', ..."
2,"['html', 'css']","['whats', 'best', 'way', 'go', 'photoshop', 'm..."
3,['language-agnostic'],"['lookup', 'data', 'book', 'barcode', 'number'..."
4,"['.net', 'com']","['embeddable', 'webkit', 'component', 'window'..."
...,...,...
38388,"['php', 'mysql']","['php', 'date', 'issue', 'daylight', 'saving',..."
38389,"['html', 'css']","['attach', 'image', 'bottom', 'middle', 'tryin..."
38390,"['c#', '.net', 'generics']","['c#', 'cant', 'liststring', 'object', 'stored..."
38391,"['html', 'xml']","['way', 'comment', 'code', 'notepad2', 'notepa..."


#### 1.1.2 Préparation Train/Tests datasets

In [9]:
def removestring(text):
    return ast.literal_eval(text)

df_lemm['TagsTokens'] = df_lemm['TagsTokens'].apply(removestring)

C:\Users\alice\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
dfnew = df_lemm.copy()

In [11]:
mlb = MultiLabelBinarizer()
ytags = mlb.fit_transform(dfnew['TagsTokens'])

In [12]:
ytags #check de la binarisation

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(dfnew['text_lemmatized'], ytags, test_size=0.2,train_size=0.8, random_state=7)

mlb.inverse_transform(y_train)

### 1.2 Extraction de l'information du texte: tf-idf

#### 1.2.1 tf-idf sur les données train

In [14]:
#initislisation du vectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf = True, max_features = 1000)
#transformation des données
X_tfidf_train = tfidf_vectorizer.fit_transform(X_train)

In [15]:
X_tfidf_train

<30714x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 956361 stored elements in Compressed Sparse Row format>

In [16]:
# récupération du premier vecteur
first_vector_tfidfvectorizer_train = X_tfidf_train[0]

In [17]:
# placement des valeurs tf-idf values dans un pandas data frame
df_lemm_tfidfd_train = pd.DataFrame(first_vector_tfidfvectorizer_train.T.todense(), 
                                    index = tfidf_vectorizer.get_feature_names(), columns = ["tfidf"])
df_lemm_tfidfd_train.sort_values(by = ["tfidf"],ascending=False)

,tfidf
map,0.717726
collection,0.404702
easy,0.261893
value,0.171563
make,0.169668
...,...
failed,0.000000
fails,0.000000
failure,0.000000
fairly,0.000000


In [18]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


#### 1.2.2 tf-idf sur les données test

In [19]:
#initislisation du vectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf = True, max_features = 1000)
#transformation des données
X_tfidf_test= tfidf_vectorizer.fit_transform(X_test)

In [20]:
X_tfidf_test

<7679x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 239032 stored elements in Compressed Sparse Row format>

In [21]:
# récupération du premier vecteur
first_vector_tfidfvectorizer_test=X_tfidf_test[0]

In [22]:
# placement des valeurs tf-idf values dans un pandas data frame
df_lemm_tfidfd_test = pd.DataFrame(first_vector_tfidfvectorizer_test.T.todense(), 
                                   index = tfidf_vectorizer.get_feature_names(), columns = ["tfidf"])
df_lemm_tfidfd_test.sort_values(by = ["tfidf"],ascending=False)

,tfidf
dialog,0.449423
assembly,0.404126
framework,0.351412
reference,0.330897
net,0.303273
...,...
fact,0.000000
factory,0.000000
fail,0.000000
failed,0.000000


### 1.3 Méthode 1: Approche non supervisée

#### 1.3.1 Latent Dirichlet Allocation (méthode non-supervisée)  -  train dataset

In [23]:
n_topics = 10

# Créer le modèle LDA
lda_model = LatentDirichletAllocation(
        n_components =n_topics, 
        max_iter = 5, 
        learning_method ='online', 
        learning_offset = 50,
        random_state = 0)

# Fitter sur les données
lda_model.fit(X_tfidf_train)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [24]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, X_tfidf_train, tfidf_vectorizer, mds='tsne')
panel

C:\Users\alice\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6     -96.589386  24.950237       1        1  24.103635
9      88.614677  17.021690       2        1  15.568372
7     -13.801842 -33.460793       3        1  15.040158
1     -11.249109 -95.517563       4        1  11.728240
3      31.267065  77.249123       5        1  10.922412
2     -77.720375 -45.640770       6        1   8.958103
0      26.660820  12.413454       7        1   6.185290
5     -33.339455  24.555094       8        1   2.628912
8      54.342808 -53.057274       9        1   2.596865
4     -41.044472  87.354141      10        1   2.268014, topic_info=    Category        Freq       Term       Total  loglift  logprob
830  Default  673.000000      speed  673.000000  30.0000  30.0000
225  Default  594.000000   database  594.000000  29.0000  29.0000
871  Default  702.000000        tab  702.000000  28.0000  28.0000
373  Default  542.000000     format  542.000000  27.0000  27.0000
783  Default  661.000000     server  661.000000  26.0000  26.0000
..       ...         ...        ...         ...      ...      ...
230  Topic10    0.380729       deal   54.193637  -1.1720  -8.9804
700  Topic10    0.380807      quite   56.472386  -1.2129  -8.9802
882  Topic10    0.388704       test  117.604733  -1.9260  -8.9597
249  Topic10    0.382514     detail   82.673702  -1.5896  -8.9757
185  Topic10    0.381164  condition   78.491468  -1.5413  -8.9792

[506 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
7         1  0.439909     18
7         2  0.135357     18
7         3  0.033839     18
7         4  0.304552     18
7         7  0.033839     18
...     ...       ...    ...
995       4  0.085987    xml
995       7  0.737035    xml
996       1  0.064618  xmlns
996       3  0.096927  xmlns
996       4  0.807728  xmlns

[1936 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 8, 2, 4, 3, 1, 6, 9, 5])

#### 1.3.2 Negative Matrix Factorisation (NMF) - train dataset

In [25]:
n_topics = 10
n_top_words = 10


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


nmf = NMF(n_components = n_topics, random_state = 0, alpha = 0.1, l1_ratio = 0.5).fit(X_tfidf_train)

print("Topics found via NMF:")
print_top_words(nmf, tfidf_feature_names, n_top_words)

Topics found via NMF:

Topic #0:
window application use code way one time java net app

Topic #1:
function var return value array javascript call foo code error

Topic #2:
table sql query column database select row data value server

Topic #3:
image page text html script javascript cs button jquery content

Topic #4:
file directory path line folder script xml command source error

Topic #5:
class public object method new property void type int private

Topic #6:
string list value character new array name return public convert

Topic #7:
service server web request aspnet error http client user application

Topic #8:
test unit run testing mock running code build fail output

Topic #9:
visual studio project build 2008 2010 assembly solution version error



### 1.4 Méthode 2: Approche purement supervisée

In [26]:
def train_classifier(X_train, y_train, X_valid=None, y_valid=None, C=1.0, model='lr'):
    """
      X_train, y_train — training data
      
      return: trained classifier
      
    """
    
    if model=='lr':
        model = LogisticRegression(C=C, penalty='l1', dual=False, solver='liblinear')
        model = OneVsRestClassifier(model)
        model.fit(X_train, y_train)
    
    elif model=='svm':
        model = LinearSVC(C=C, penalty='l1', dual=False, loss='squared_hinge')
        model = OneVsRestClassifier(model)
        model.fit(X_train, y_train)
    
    elif model=='nbayes':
        model = MultinomialNB(alpha=0.3)
        model = OneVsRestClassifier(model)
        model.fit(X_train, y_train)
        
    elif model=='mlp':
        model = MLPClassifier()
        model = OneVsRestClassifier(model)
        model.fit(X_train, y_train)

    return model


#### 1.4.1 Réression logistique

In [27]:
%time classifier_tfidf = train_classifier(X_tfidf_train, y_train, C=1.0, model='lr')

Wall time: 22.3 s


In [28]:
y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_tfidf_test)

In [29]:
y_test_pred_inversed = mlb.inverse_transform(y_test_predicted_labels_tfidf)
y_test_inversed = mlb.inverse_transform(y_test)
for i in range(20):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_test.iloc[i],
        ','.join(y_test_inversed[i]),
        ','.join(y_test_pred_inversed[i])
    ))

Title:	['assembly', 'add', 'reference', 'dialog', 'differ', 'depending', 'net', 'framework', 'used', 'curious', 'listing', 'assembly', 'add', 'reference', 'dialog', 'vary', 'depending', 'version', 'net', 'framework', 'used', 'selected', 'project', 'differ', 'love', 'difference', 'lie', 'thanks']
True labels:	.net,c#,visual-studio
Predicted labels:	


Title:	['check', 'socket', 'still', 'open', 'c++', 'app', 'us', 'standard', 'socket', 'call', 'want', 'tell', 'socket', 'still', 'open', 'without', 'sending', 'receiving', 'data', 'reliable', 'select', 'ioctlsocket', 'call', 'make']
True labels:	c++
Predicted labels:	


Title:	['array', 'case', '5', '5', 'joel', 'point', 'stack', 'overflow', 'podcast', '#', '34', 'c', 'programming', 'language', 'aka', 'k', 'r', 'mention', 'property', 'array', 'c', '5', '5', 'joel', 'say', 'pointer', 'arithmetic', 'still', 'understand', '5', '5']
True labels:	arrays
Predicted labels:	


Title:	['class', 'net', 'application', 'internal', 'note', 'application

#### 1.4.2 SVM

In [30]:
%time classifier_tfidf = train_classifier(X_tfidf_train, y_train, C=1.0, model='svm')

Wall time: 2min 5s


In [31]:
y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_tfidf_test)

In [32]:
y_test_pred_inversed = mlb.inverse_transform(y_test_predicted_labels_tfidf)
y_test_inversed = mlb.inverse_transform(y_test)
for i in range(20):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_test.iloc[i],
        ','.join(y_test_inversed[i]),
        ','.join(y_test_pred_inversed[i])
    ))

Title:	['assembly', 'add', 'reference', 'dialog', 'differ', 'depending', 'net', 'framework', 'used', 'curious', 'listing', 'assembly', 'add', 'reference', 'dialog', 'vary', 'depending', 'version', 'net', 'framework', 'used', 'selected', 'project', 'differ', 'love', 'difference', 'lie', 'thanks']
True labels:	.net,c#,visual-studio
Predicted labels:	


Title:	['check', 'socket', 'still', 'open', 'c++', 'app', 'us', 'standard', 'socket', 'call', 'want', 'tell', 'socket', 'still', 'open', 'without', 'sending', 'receiving', 'data', 'reliable', 'select', 'ioctlsocket', 'call', 'make']
True labels:	c++
Predicted labels:	


Title:	['array', 'case', '5', '5', 'joel', 'point', 'stack', 'overflow', 'podcast', '#', '34', 'c', 'programming', 'language', 'aka', 'k', 'r', 'mention', 'property', 'array', 'c', '5', '5', 'joel', 'say', 'pointer', 'arithmetic', 'still', 'understand', '5', '5']
True labels:	arrays
Predicted labels:	


Title:	['class', 'net', 'application', 'internal', 'note', 'application

#### 1.4.3 Naive Bayes

In [33]:
%time classifier_tfidf = train_classifier(X_tfidf_train, y_train, C=1.0, model='nbayes')

Wall time: 1.16 s


In [34]:
y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_tfidf_test)

In [35]:
y_test_pred_inversed = mlb.inverse_transform(y_test_predicted_labels_tfidf)
y_test_inversed = mlb.inverse_transform(y_test)
for i in range(20):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_test.iloc[i],
        ','.join(y_test_inversed[i]),
        ','.join(y_test_pred_inversed[i])
    ))

Title:	['assembly', 'add', 'reference', 'dialog', 'differ', 'depending', 'net', 'framework', 'used', 'curious', 'listing', 'assembly', 'add', 'reference', 'dialog', 'vary', 'depending', 'version', 'net', 'framework', 'used', 'selected', 'project', 'differ', 'love', 'difference', 'lie', 'thanks']
True labels:	.net,c#,visual-studio
Predicted labels:	


Title:	['check', 'socket', 'still', 'open', 'c++', 'app', 'us', 'standard', 'socket', 'call', 'want', 'tell', 'socket', 'still', 'open', 'without', 'sending', 'receiving', 'data', 'reliable', 'select', 'ioctlsocket', 'call', 'make']
True labels:	c++
Predicted labels:	


Title:	['array', 'case', '5', '5', 'joel', 'point', 'stack', 'overflow', 'podcast', '#', '34', 'c', 'programming', 'language', 'aka', 'k', 'r', 'mention', 'property', 'array', 'c', '5', '5', 'joel', 'say', 'pointer', 'arithmetic', 'still', 'understand', '5', '5']
True labels:	arrays
Predicted labels:	


Title:	['class', 'net', 'application', 'internal', 'note', 'application

#### 1.4.4 Multi-Layered Preceptron - Neural Networks

In [36]:
%time classifier_tfidf = train_classifier(X_tfidf_train, y_train, C=1.0, model='mlp')

Wall time: 50min 37s


In [37]:
y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_tfidf_test)

In [38]:
y_test_pred_inversed = mlb.inverse_transform(y_test_predicted_labels_tfidf)
y_test_inversed = mlb.inverse_transform(y_test)
for i in range(20):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_test.iloc[i],
        ','.join(y_test_inversed[i]),
        ','.join(y_test_pred_inversed[i])
    ))

Title:	['assembly', 'add', 'reference', 'dialog', 'differ', 'depending', 'net', 'framework', 'used', 'curious', 'listing', 'assembly', 'add', 'reference', 'dialog', 'vary', 'depending', 'version', 'net', 'framework', 'used', 'selected', 'project', 'differ', 'love', 'difference', 'lie', 'thanks']
True labels:	.net,c#,visual-studio
Predicted labels:	


Title:	['check', 'socket', 'still', 'open', 'c++', 'app', 'us', 'standard', 'socket', 'call', 'want', 'tell', 'socket', 'still', 'open', 'without', 'sending', 'receiving', 'data', 'reliable', 'select', 'ioctlsocket', 'call', 'make']
True labels:	c++
Predicted labels:	


Title:	['array', 'case', '5', '5', 'joel', 'point', 'stack', 'overflow', 'podcast', '#', '34', 'c', 'programming', 'language', 'aka', 'k', 'r', 'mention', 'property', 'array', 'c', '5', '5', 'joel', 'say', 'pointer', 'arithmetic', 'still', 'understand', '5', '5']
True labels:	arrays
Predicted labels:	events,iphone


Title:	['class', 'net', 'application', 'internal', 'note',